In [1]:
import os
import pandas as pd
from datetime import date, timedelta, datetime
from sqlalchemy import create_engine, text
from pandas.tseries.offsets import BDay

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()
engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()
engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

today = date.today()
today

datetime.date(2025, 8, 8)

In [3]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2025-08-08
yesterday: 2025-08-07


In [5]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\Downloads\Datasets
dts_path = os.path.join(user_path, "Downloads", "Datasets")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")
#C:\Users\PC1\OneDrive\A5\Excel
xsl_path = os.path.join(base_path, "Excel")

In [7]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path (dat_path): {dat_path}") 
print(f"Excel path (xsl_path): {xsl_path}") 
print(f"Google Drive path (god_path): {god_path}")
print(f"iCloudDrive path (icd_path): {icd_path}") 
print(f"Obsidian path (osd_path): {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Daily
Base path: C:\Users\PC1\OneDrive\A5
Data path (dat_path): C:\Users\PC1\OneDrive\A5\Data
Excel path (xsl_path): C:\Users\PC1\OneDrive\A5\Excel
Google Drive path (god_path): C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path (icd_path): C:\Users\PC1\iCloudDrive\Data
Obsidian path (osd_path): C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


### Tables in the process

In [10]:
format_dict = {
    'shares':'{:,}',    
    'price':'{:.2f}',
    'dividend':'{:.4f}', 
    'date':'{:%Y-%m-%d}', 
    
    'qty':'{:,}','shares':'{:,}',
    'price':'{:.2f}','buy_price':'{:.2f}',
    'dividend':'{:.4f}',    
    'fee':'{:,.2f}','vat':'{:,.2f}','net':'{:,.2f}',
  
    'days':'{:,}',
    'price':'{:.2f}',
    'fee':'{:,.2f}','vat':'{:,.2f}','net':'{:,.2f}','profit':'{:,.2f}',
    'percent':'{:,.2f}%','yearly':'{:,.2f}%',   
    
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    }

In [71]:
# Sells table in MySQL portfolio database
sql = """
SELECT name, stock_id, B.date AS buy_date, qty, B.price AS buy_price, S.* 
FROM sells S
JOIN buys B ON S.buy_id = B.id
JOIN stocks T ON B.stock_id = T.id
ORDER BY S.id DESC
LIMIT 1"""
sells = pd.read_sql(sql, conpf)
sells.style.format(format_dict)

,name,stock_id,buy_date,qty,buy_price,id,buy_id,date,price,fee,vat,net,days,profit,percent,yearly,sequence,chart,dividend_id
0,GVREIT,169,2023-09-22,"6,000",6.10,1106,1287,2025-08-08,6.70,83.21,5.82,"40,110.97",686,"3,429.91",9.35%,4.98%,-106,None,0


### In case of sale, retreive buy id from sells record (buy_id)

In [74]:
# Buys table in MySQL portfolio database
transaction = 'S'

names = sells['name']
name = names.to_string(index=False)
name

'GVREIT'

In [76]:
sr_qty = sells['qty']
qty = sr_qty.to_string(index=False)
qty = int(qty) * -1

sr_price = sells['buy_price']
buy_price = sr_price.to_string(index=False)
buy_price = float(buy_price)

sr_price = sells['price']
sell_price = sr_price.to_string(index=False)
sell_price = float(sell_price)

buy_ids = sells['buy_id']
buy_id = buy_ids.to_string(index=False)

sql = """
SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
WHERE B.id = %s
"""
sql = sql % buy_id
print(sql)

buys = pd.read_sql(sql, conpf)
buys.style.format(format_dict)


SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
WHERE B.id = 1287



,name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,GVREIT,1287,169,2023-09-22,"6,000",6.10,75.76,5.30,Sold,"36,681.06",HD,None


In [78]:
transaction, name, qty, sell_price, buy_price

('S', 'GVREIT', -6000, 6.7, 6.1)

### In case of buy, retreive buy id from latest buy record (id)

In [56]:
sql = """
SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
ORDER BY id DESC
LIMIT 1
"""
print(sql)
buys = pd.read_sql(sql, conpf)
buys.style.format(format_dict)


SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
ORDER BY id DESC
LIMIT 1



,name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,PTT,1336,118,2025-08-08,"2,500",32.00,165.60,11.59,Active,"80,177.19",HD,None


In [58]:
transaction = 'B'

names = buys['name']
name = names.to_string(index=False)

sr_qty = buys['qty']
qty = sr_qty.to_string(index=False)
qty = int(qty)

sr_price = buys['price']
buy_price = sr_price.to_string(index=False)
buy_price = float(buy_price)

transaction, name, qty, buy_price

('B', 'PTT', 2500, 32.0)

## Begin of Sale transaction

### Begin of Update buy table in MySQL stock database from sale transaction

In [80]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal','dividend'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'GVREIT'



,name,date,shares,price,active,period,grade
0,GVREIT,2022-08-24,"75,000",7.64,1,2,C1


In [82]:
print(transaction, name, qty, buy_price)

S GVREIT -6000 6.1


In [84]:
new_qty = buy['shares'] + qty
print(new_qty.iloc[0])

69000


In [86]:
new_price = 7.75
new_price

7.75

In [88]:
def update_buy(const, name, new_qty, new_price):
    # Extract scalar values from Pandas Series
    new_qty_value = int(new_qty.iloc[0])
    
    # Use parameterized query to avoid SQL injection
    sqlUpd = text("""
        UPDATE buy
        SET volbuy = :new_qty,
        price = :new_price
        WHERE name = :name
    """)
    
    # Execute the query with parameters
    rp = const.execute(sqlUpd, {
        'new_qty': new_qty_value,
        'new_price': new_price,
        'name': name
    })

    return f"Records updated = {rp.rowcount}"

In [90]:
# Example usage
update_buy(const, name, new_qty, new_price)

'Records updated = 1'

In [92]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal','dividend'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'GVREIT'



,name,date,shares,price,active,period,grade
0,GVREIT,2022-08-24,"69,000",7.75,1,2,C1


In [127]:
# Define the SQL query
sqlDel = text("""
    DELETE FROM buy
    WHERE name = :name AND volbuy = 0;
""")

# Execute the query with the correct parameter dictionary
result = const.execute(sqlDel, {'name': name})

# Print the number of rows deleted
print(f"Records deleted: {result.rowcount}")  

Records deleted: 0


### End of Update buy table in MySQL stock database from sale transaction

### Begin of Update dividend table in MySQL stock database from sale transaction

In [94]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'GVREIT'



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,GVREIT,0.2044,0.1911,0.1984,0.2050,0.7989,"75,000",2025-05-28,2025-06-12,,0


In [96]:
new_qty

0    69000
Name: shares, dtype: int64

In [98]:
def update_dividend(const, name, new_qty):
    # Extract scalar values from Pandas Series
    new_qty_value = int(new_qty.iloc[0])
    # Use parameterized query to avoid SQL injection
    sqlUpd = text("""
        UPDATE dividend
        SET shares = :new_qty
        WHERE name = :name
    """)
    # Execute the query with parameters
    rp = const.execute(sqlUpd, {
        'new_qty': new_qty_value,
        'name': name
    })
    return f"Records updated = {rp.rowcount}"

In [100]:
update_dividend(const, name, new_qty)

'Records updated = 1'

In [102]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'GVREIT'



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,GVREIT,0.2044,0.1911,0.1984,0.2050,0.7989,"69,000",2025-05-28,2025-06-12,,0


In [45]:
# Define the SQL query
sqlDel = text("""
    DELETE FROM dividend
    WHERE name = :name AND shares = 0;
""")

# Execute the query with the correct parameter dictionary
result = const.execute(sqlDel, {'name': name})

# Print the number of rows deleted
print(f"Records deleted: {result.rowcount}")  

Records deleted: 0


### End of Update dividend table in MySQL stock database from sale transaction

### Begin of Update stocks table in SQLite port_lite database from sale transaction

In [104]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stock = pd.read_sql(sql, conlite)
stock.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'GVREIT'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,108,GVREIT,0.00,0.00,B,6.20,7.65,0.00,0.00,7.64,"5,000",-4,4,"75,000",0.000000,0.000000,RND20Pct,SET


In [106]:
# Extract scalar values from Pandas Series (if applicable)
new_qty_value = new_qty.iloc[0] if hasattr(new_qty, 'iloc') else new_qty
#cost_value = cost.iloc[0] if hasattr(cost, 'iloc') else cost
# Print the values
print(name, transaction, new_qty_value, new_price)

GVREIT S 69000 7.75


In [51]:
#print("Raw cost value before extraction:", cost)

In [108]:
def update_stock(conlite, name, new_qty, new_price):
    # Extract scalar values from Pandas Series
    new_qty_value = int(new_qty.iloc[0])  
    # Use parameterized query to avoid SQL injection
    sqlUpd = text("""
        UPDATE stocks
        SET available_qty = :new_qty,
        cost = :new_price
        WHERE name = :name
    """)    
    # Execute the query with parameters
    rp = conlite.execute(sqlUpd, {
        'new_qty': new_qty_value, # Use the scalar value
        'new_price': new_price,
        'name': name
    })
    return f"Records updated = {rp.rowcount}"

In [110]:
# Example usage
update_stock(conlite, name, new_qty, new_price)

'Records updated = 1'

In [112]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'GVREIT'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,108,GVREIT,0.00,0.00,B,6.20,7.65,0.00,0.00,7.75,"5,000",-4,4,"69,000",0.000000,0.000000,RND20Pct,SET


In [103]:
# Define the SQL query
sqlDel = text("""
    DELETE FROM stocks
    WHERE name = :name AND available_qty = 0;
""")

# Execute the query with the correct parameter dictionary
result = conlite.execute(sqlDel, {'name': name})

# Print the number of rows deleted
print(f"Records deleted: {result.rowcount}")  

Records deleted: 0


### End of Update stocks table in SQLite port_lite database from sale transaction

## End of Sale process

### =====================================================

## Begin of  Buy process

### Begin of Update buy table in MySQL database from buy transaction

In [60]:
sql = """
SELECT *, volbuy * price AS cost_amt 
FROM buy
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy['date'] = pd.to_datetime(buy['date'])
buy.drop(['volsell', 'volbal'], axis=1, inplace = True)
buy.rename(columns={'volbuy':'shares'}, inplace = True)
buy['shares'] = buy['shares'].astype('int64')
buy


SELECT *, volbuy * price AS cost_amt 
FROM buy
WHERE name = 'PTT'



,name,date,shares,price,active,dividend,period,grade,cost_amt


In [62]:
print(transaction, name, qty, buy_price)

B PTT 2500 32.0


In [64]:
new_unit_cost = buy['cost_amt'].iloc[0] + (qty * buy_price)
new_qty = buy['shares'].iloc[0] + qty
print(new_unit_cost)
print(new_qty)
new_unit_cost = new_unit_cost / new_qty
print(new_unit_cost)

IndexError: single positional indexer is out-of-bounds

In [25]:
new_unit_cost = 10.75
print(new_unit_cost)

10.75


### In case there is the same stock name in port

In [23]:
def update_buy(const, name, new_qty, new_unit_cost):
    # Extract scalar values from Pandas Series
    new_qty_value = new_qty
    new_unit_cost_value = new_unit_cost
    
    # Use parameterized query to avoid SQL injection
    sqlUpd = text("""
        UPDATE buy
        SET volbuy = :new_qty, 
            price = :new_unit_cost
        WHERE name = :name
    """)
    
    # Execute the query with parameters
    rp = const.execute(sqlUpd, {
        'new_qty': new_qty_value,  # Use the scalar value
        'new_unit_cost': new_unit_cost_value,  # Use the scalar value
        'name': name
    })

    return f"Records updated = {rp.rowcount}"

In [27]:
# Example usage
update_buy(const, name, new_qty, new_unit_cost)

'Records updated = 1'

In [29]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'AIMIRT'



,name,date,shares,price,active,dividend,period,grade
0,AIMIRT,2023-08-17,"15,000",10.75,1,0.7883,2,A1


### In case of totally new stock

In [74]:
rcd = {
    'name': 'PTT',
    'date': '2025-08-08',
    'volbuy': 2500,
    'price': 32,
    'volsell': 0,
    'volbal': 0,
    'active': 1,
    'dividend': 0,
    'period': 2,
    'grade': 'A2'
}
sqlIns = text("""
INSERT INTO buy (name, date, volbuy, price, volsell, volbal, active, dividend, period, grade)
VALUES (:name, :date, :volbuy, :price, :volsell, :volbal, :active, :dividend, :period, :grade)
""")
rp = const.execute(sqlIns, rcd)
rp.rowcount

1

In [76]:
name = 'PTT'

In [78]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'PTT'



,name,date,shares,price,active,dividend,period,grade
0,PTT,2025-08-08,"2,500",32.00,1,0.0000,2,A2


### End of Update buy table in MySQL database from buy transaction

### Begin of Update dividend table in MySQL stock database from buy transaction

In [80]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s' 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'PTT' 



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,PTT,1.3000,0.0000,0.8000,0.0000,2.1000,0,2025-03-06,2025-04-29,,0


In [82]:
new_qty = dividend['shares'].iloc[0] + qty
print(new_qty)

2500


In [84]:
def update_dividend(const, name, new_qty):
    # Extract scalar values from Pandas Series
    new_qty_value = new_qty  
    # Use parameterized query to avoid SQL injection
    sqlUpd = text("""
        UPDATE dividend
        SET shares = :new_qty
        WHERE name = :name
    """)
    
    # Execute the query with parameters
    rp = const.execute(sqlUpd, {
        'new_qty': new_qty_value,  # Use the scalar value
        'name': name
    })

    return f"Records updated = {rp.rowcount}"

In [86]:
update_dividend(const, name, new_qty)

'Records updated = 1'

In [88]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s' 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'PTT' 



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,PTT,1.3000,0.0000,0.8000,0.0000,2.1000,"2,500",2025-03-06,2025-04-29,,0


### In case of totally new dividend

In [15]:
rcd = ['TVO', 0.25, 0, 0.33, 0, 0.58, 1000, '2023-08-30', '2023-09-14', '', 1]
rcd

['TOA', 0.25, 0, 0.33, 0, 0.58, 1000, '2023-08-30', '2023-09-14', '', 1]

In [15]:
sqlIns = """
INSERT INTO dividend (name, q4, q3, q2, q1, dividend, shares, xdate, paiddate, kind, actual)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
print(sqlIns)


INSERT INTO dividend (name, q4, q3, q2, q1, dividend, shares, xdate, paiddate, kind, actual)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)



In [16]:
rp = const.execute(sqlIns, rcd)
rp.rowcount

1

In [17]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s' 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'TOA' 



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,TOA,0.2500,0.0000,0.3300,0.0000,0.5800,"1,000",2023-08-30,2023-09-14,,1


### End of Create dividend table in MySQL stock database from buy transaction

### Begin of Update stocks table in SQLite port_lite database from buy transaction

In [90]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stock = pd.read_sql(sql, conlite)
stock


SELECT * 
FROM stocks 
WHERE name = 'PTT'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market


In [44]:
#
new_buy_target = 9.8
new_sell_target = 10.75
new_buy_qty = 2_500
print(transaction, name, qty, buy_price, new_buy_target, new_sell_target, new_buy_qty) 

B AIMIRT 2500 9.9 9.8 10.75 2500


In [46]:
print(new_qty)

15000


In [48]:
print(new_unit_cost)

10.75


In [50]:
def update_stock(conlite, name, new_qty, new_unit_cost, new_buy_target, new_sell_target, new_buy_qty):
    # Extract scalar values from Pandas Series
    new_qty_value = int(new_qty)
    new_unit_cost_value = float(new_unit_cost)   
    # Use parameterized query to avoid SQL injection
    sqlUpd = text("""
        UPDATE stocks
        SET available_qty = :new_qty, 
            cost = :new_unit_cost,
            buy_target = :new_buy_target,
            sell_target = :new_sell_target,    
            qty = :new_buy_qty
        WHERE name = :name
    """)    
    # Execute the query with parameters
    rp = conlite.execute(sqlUpd, {
        'new_qty': new_qty_value,  # Use the scalar value
        'new_unit_cost': new_unit_cost_value, # Use the scalar value 
        'new_buy_target': new_buy_target,
        'new_sell_target': new_sell_target,     
        'new_buy_qty': new_buy_qty,
        'name': name
    })
    return f"Records updated = {rp.rowcount}"

In [52]:
# Example usage
update_stock(conlite, name, new_qty, new_unit_cost, new_buy_target, new_sell_target, new_buy_qty)

'Records updated = 1'

In [54]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stock = pd.read_sql(sql, conlite)
stock


SELECT * 
FROM stocks 
WHERE name = 'AIMIRT'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,101,AIMIRT,0.0,0.0,S,9.8,10.75,0.0,0.0,10.75,2500,-4,4,15000,0.0,0.0,RND15pct,SET


### End of Update stocks table in SQLite port_lite database from buy transaction

## End of  Buy process

### =================================================

In [114]:
#price_date = '2025-02-14'
sql = "SELECT period, buy.grade AS grade, buy.name AS name, buy.date AS date, \
FORMAT(volbuy,0) AS volbuy, FORMAT(buy.price,2) AS buy_price, price.price AS mkt_price,\
FORMAT((volbuy * buy.price),2) AS amtbuy, FORMAT((volbuy * price.price),2) AS amtmkt, \
FORMAT(((price.price - buy.price) * volbuy),2) AS amtpol, \
FORMAT((((price.price - buy.price)*volbuy)/(volbuy*buy.price)*100),2) AS pctpol \
FROM buy INNER JOIN price ON buy.name = price.name WHERE price.date = '%s' \
ORDER BY period, buy.name"
sql = sql % (yesterday)
print(sql)

SELECT period, buy.grade AS grade, buy.name AS name, buy.date AS date, FORMAT(volbuy,0) AS volbuy, FORMAT(buy.price,2) AS buy_price, price.price AS mkt_price,FORMAT((volbuy * buy.price),2) AS amtbuy, FORMAT((volbuy * price.price),2) AS amtmkt, FORMAT(((price.price - buy.price) * volbuy),2) AS amtpol, FORMAT((((price.price - buy.price)*volbuy)/(volbuy*buy.price)*100),2) AS pctpol FROM buy INNER JOIN price ON buy.name = price.name WHERE price.date = '2025-08-07' ORDER BY period, buy.name


In [116]:
output = pd.read_sql(sql, const)
output

,period,grade,name,date,volbuy,buy_price,mkt_price,amtbuy,amtmkt,amtpol,pctpol
0,1,C1,AH,2023-06-08,"1,200",37.00,15.60,"44,400.00","18,720.00","-25,680.00",-57.84
1,1,A1,AWC,2023-06-15,"9,000",4.96,2.34,"44,640.00","21,060.00","-23,580.00",-52.82
2,1,A2,JMART,2023-01-20,"6,800",33.00,9.20,"224,400.00","62,560.00","-161,840.00",-72.12
3,1,C2,ORI,2022-04-07,"50,000",9.00,2.14,"450,000.00","107,000.00","-343,000.00",-76.22
4,1,C2,PTTGC,2021-03-17,"6,000",64.75,24.20,"388,500.00","145,200.00","-243,300.00",-62.63
5,1,C2,SCC,2021-09-20,600,405.00,203.00,"243,000.00","121,800.00","-121,200.00",-49.88
6,1,A3,SINGER,2023-01-19,"6,000",24.80,5.80,"148,800.00","34,800.00","-114,000.00",-76.61
7,2,A1,AIMIRT,2023-08-17,"15,000",10.75,9.85,"161,250.00","147,750.00","-13,500.00",-8.37
8,2,C1,CPNREIT,2022-08-16,"55,000",18.00,11.50,"990,000.00","632,500.00","-357,500.00",-36.11
9,2,C1,DIF,2020-08-01,"40,000",13.10,8.40,"524,000.00","336,000.00","-188,000.00",-35.88


In [118]:
file_name = 'portfolio.csv'
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [120]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file : C:\Users\PC1\OneDrive\A5\Data\portfolio.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\portfolio.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\portfolio.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\portfolio.csv


In [122]:
output.to_csv(output_file, header=True, index=False)
output.to_csv(icd_file, header=True, index=False)
output.to_csv(god_file, header=True, index=False)
output.to_csv(osd_file, header=True, index=False)

In [124]:
file_name = '035-portfolio.xlsx'
xsl_file = os.path.join(xsl_path, file_name)

output.to_excel(xsl_file, index=False)